Random Forest is constructed using multiple decision trees and the final decision is obtained by Majority votes of decision trees

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = sns.load_dataset("penguins")

In [3]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [4]:
df.shape

(344, 7)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [6]:
df.isnull().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

Drop the null values

In [7]:
df.dropna(inplace=True)

In [8]:
df.isnull().sum()


species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

Feature Engineering

One Hot Encoding : Trnasforming Categorical data into Numeric

In [9]:
df.sex.unique()

array(['Male', 'Female'], dtype=object)

In [10]:
pd.get_dummies(df['sex']).astype(int).head()

,Female,Male
0,0,1
1,1,0
2,1,0
4,1,0
5,0,1


In [11]:
sex = pd.get_dummies(df['sex'], drop_first=True).astype(int)
sex.head()

,Male
0,1
1,0
2,0
4,0
5,1


In [12]:
df.island.unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [13]:
pd.get_dummies(df['island']).astype(int).head()

,Biscoe,Dream,Torgersen
0,0,0,1
1,0,0,1
2,0,0,1
4,0,0,1
5,0,0,1


In [14]:
island = pd.get_dummies(df['island'], drop_first=True).astype(int)

Concatenate the above two dataframes to the original df

In [15]:
new_data = pd.concat([df, island, sex], axis=1)

In [16]:
new_data.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,Dream,Torgersen,Male
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,0,1,1
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,0,1,0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,0,1,0
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,0,1,0
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male,0,1,1


In [17]:
# drop the repeated columns
new_data.drop(['sex', 'island'], axis=1, inplace=True);
new_data.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Dream,Torgersen,Male
0,Adelie,39.1,18.7,181.0,3750.0,0,1,1
1,Adelie,39.5,17.4,186.0,3800.0,0,1,0
2,Adelie,40.3,18.0,195.0,3250.0,0,1,0
4,Adelie,36.7,19.3,193.0,3450.0,0,1,0
5,Adelie,39.3,20.6,190.0,3650.0,0,1,1


Creating Separate Target Variable

In [18]:
y = new_data.species
y.head()

0    Adelie
1    Adelie
2    Adelie
4    Adelie
5    Adelie
Name: species, dtype: object

In [19]:
y.unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [20]:
# converting species into numeric

y = y.map({'Adelie':0, 'Chinstrap':1, 'Gentoo':2})

In [21]:
y.head()

0    0
1    0
2    0
4    0
5    0
Name: species, dtype: int64

Dropping the Target Variable : Species

In [22]:
new_data.drop('species', inplace=True, axis=1)

In [23]:
new_data.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,Dream,Torgersen,Male
0,39.1,18.7,181.0,3750.0,0,1,1
1,39.5,17.4,186.0,3800.0,0,1,0
2,40.3,18.0,195.0,3250.0,0,1,0
4,36.7,19.3,193.0,3450.0,0,1,0
5,39.3,20.6,190.0,3650.0,0,1,1


In [24]:
X = new_data

Splitting the Dataset into Training and Test data

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [27]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(233, 7) (100, 7) (233,) (100,)


Training Random Forest Classification on Training Set

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
classifier = RandomForestClassifier(n_estimators=5, criterion='entropy', random_state=0)

In [30]:
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=5, random_state=0)

Predicting the Test Results

In [31]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 2, 0, 1, 0, 2, 2, 1, 1, 1, 0, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0, 0,
       2, 1, 0, 0, 2, 2, 1, 2, 1, 2, 0, 0, 2, 2, 1, 2, 0, 0, 0, 0, 1, 1,
       0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 1, 1, 2, 2, 2, 0, 0, 2,
       0, 2, 0, 2, 0, 0, 2, 2, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2,
       1, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0])

Confusion Matrix

In [32]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

In [33]:
cm = confusion_matrix(y_test, y_pred)

In [34]:
print(cm)

[[47  1  0]
 [ 1 17  0]
 [ 0  0 34]]


In [35]:
accuracy_score(y_test, y_pred)

0.98

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        48
           1       0.94      0.94      0.94        18
           2       1.00      1.00      1.00        34

    accuracy                           0.98       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.98      0.98      0.98       100



Lets change the criterion and number of trees

In [38]:
classifier = RandomForestClassifier(n_estimators=7, criterion='gini', random_state=0)

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

print(accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print(cm)

0.98
[[47  1  0]
 [ 1 17  0]
 [ 0  0 34]]
